# Chapter13. 연산자 오버로딩: 제대로하기

연산자 오버로딩은 사용자 정의 객체가 `+` 와 `|` 같은 연산자 ,`-`와 `~` 같은 단항 연산자를 사용할 수 있게 해준다. 파이썬에서는 여기에서 더 나아가 함수 호출`()`, 속성 접근 `.`, 항목 접근/슬라이싱`[]`도 연산자로 구현되어 있지만, 이 장에서는 단항 연산자와 중위 연산자만 다룬다.

## 13.1 연산자 오버로딩 기본 지식
연산자 오버로딩을 혐오하는 사람도 많다. 이 언어 기능은 남용되거나, 프로그래머를 혼란스럽게 만들거나, 버그를 만들거나, 예상치 못한 성능상의 병목이 될 수도 있다. 파이썬은 다음과 같은 제한을 두어 융통성, 사용성, 안전성을 적절히 유지한다.
- 내장 자료형에 대한 연산자는 오버로딩할 수 없다.
- 새로운 연산자를 생성할 수 없으며, 기존 연산자를 오버로딩만 할 수 있다.
- is, and, or, not 연산자는 오버로딩할 수 없다. (그러나 &, |, ~ 같은 비트 연산자는 가능하다.)

In [28]:
# Vector v6
def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

def __neg__(self):
    """단항 산술 부정(`-`). x가 -2면, -x는 2다."""
    return Vector2d(-x for x in self)

def __pos__(self):
    """단항 산술 덧셈(`+`). 일반적으로 x와 -x는 동일하지만, 그렇지 않은 경우도 있다. 이 절 끝에 나오는 'x와 +x가 동일하지 않은 경우' 글상자에서 자세히 설명한다."""
    return Vector2d

In [38]:
# x와 +x가 동일하지 않은 경우
import decimal

ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
print(one_third)
print(one_third == +one_third)

ctx.prec = 28
print(one_third == +one_third)
print(+one_third)
print(one_third)

0.3333333333333333333333333333333333333333
True
False
0.3333333333333333333333333333
0.3333333333333333333333333333333333333333


In [39]:
from collections import Counter


ct = Counter('abracadabra')
ct


Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [40]:
ct['r'] = -3
ct['d'] = 0
ct

Counter({'a': 5, 'b': 2, 'r': -3, 'c': 1, 'd': 0})

In [41]:
+ct # Counter는 음수가 될 수 없으므로, 음수와 0을 제한다 (0은 왜?) => 어쨌든, 새로운 객체를 생성함

Counter({'a': 5, 'b': 2, 'c': 1})

## 13.3 벡터를 더하기 위해 + 오버로딩하기

In [6]:
from Vector.Vector_v6 import Vector

v1 = Vector([3,4,5])
v2 = Vector([6,7,8])

v1 + v2

Vector([9.0, 11.0, 13.0])

In [7]:
v1 + v2 == Vector([3+6, 4+7, 5+8])

True

In [8]:
v1 = Vector([3,4,5,6])
v3 = Vector([1,2])
v1 + v3

Vector([4.0, 6.0, 5.0, 6.0])

```python
    pairs = itertools.zip_longest(self, other, fillvalue=0.0)
    return Vector(a + b for a, b in pairs)
```
- pairs는 self에서 a를, other에서 b를 가져와 (a, b)튜플을 생성하는 제너레이터다. self와 other의 길이가 다른 경우에는 짧은 쪽 반복형의 빠진 값을 fillvalue로 채운다.
- pairs 양쪽 항목의 합을 생성하는 제너레이터 표현식을 이용해서 새로운 Vector 객체를 생성한다.

In [9]:
v1 = Vector([3,4,5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [10]:
v1 = Vector([3,4,5])
(10, 20, 30) + v1

TypeError: can only concatenate tuple (not "Vector") to tuple

In [15]:
# __radd__를 구현해주면 됨 
def __radd__(self, other):
    return self + other

## 13.4 벡터를 스칼라와 곱하기 위해 * 오버로딩하기

In [17]:
v1 = Vector([1,2,3])
v1 * 10

TypeError: unsupported operand type(s) for *: 'Vector' and 'int'

In [3]:
from Vector.Vector_v7 import Vector

In [4]:
v1 = Vector([1,2,3])
v1 * 10

Vector([10.0, 20.0, 30.0])

In [5]:
11 * v1

Vector([11.0, 22.0, 33.0])

In [6]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [7]:
from fractions import Fraction
v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

## 13.5 향상된 비교 연산자
파이썬 인터프리터가 `==`, `!=`, `>`, `<`, `>=`, `<=` 비교 연산자를 다루는 방법은 앞에서 설명한 방법과 비슷하지만(먼저 정방향 메서드를 실행하고, NotImplemented가 반환되면 역순 메서드를 실행한다.)

In [9]:
def __eq__(self, other):
    return (len(self) == len(other) and
            all(a == b for a, b in zip(self, other)))

In [11]:
va = Vector([1, 2, 3])
vb = Vector(range(1, 4))
va == vb

True

In [15]:
t3 = (1, 2, 3)
va == t3

True

In [16]:
from Vector.Vector_v8 import Vector
va = Vector([1, 2, 3])
va == t3

False

## 13.6 복합 할당 연산자
우리가 구현한 Vector는 이미 +=과 *= 복합 할당 연산자를 지원하고 있다.

In [17]:
v1 = Vector([1, 2, 3])
v1_alias = v1
print(id(v1))
v1 += Vector([4, 5, 6])
print(v1)
print(id(v1))

4380448080
(5.0, 7.0, 9.0)
4380448800


In [18]:
print(v1_alias)
v1 *= 11
print(v1)
print(id(v1))

(1.0, 2.0, 3.0)
(55.0, 77.0, 99.0)
4380447504


In [1]:
from bingoaddable import AddableBingoCage

vowels = 'AEIOU'
globe = AddableBingoCage(vowels)
globe.inspect()

('A', 'E', 'I', 'O', 'U')

In [2]:
globe.pick() in vowels

True

In [3]:
len(globe.inspect())

4

In [6]:
globe2 = AddableBingoCage('XYZ')
globe3 = globe + globe2
len(globe3.inspect())

7

In [10]:
globe3.inspect()

('A', 'E', 'O', 'U', 'X', 'Y', 'Z')

In [8]:
void = globe + [10, 20]

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [11]:
globe += 1

TypeError: right operand in += must be 'AddableBingoCage' or and iterable